In [3]:
import numpy as np
import xarray as xr

from tools.utils import xy2ll, directory_spider
from tools.flexural_fitting import deflection, BOUNDS, fitting_REMA_dh

import matplotlib.pyplot as plt
from cmcrameri import cm as cmc
from tools.plot_tools import plot_latlon_lines, shifted_colormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from matplotlib import gridspec

In [4]:
import plotSettings
plotSettings.paper()
COLS = plotSettings.COLS

In [5]:
%matplotlib tk

In [ ]:
# general data directory
data_dir = '/'

In [6]:
# all REMA data
REMA_dir = os.path.join(data_dir, 'data/REMA')
LagrangianDiff_file = 'LagrangianDifferencing/lagrangian_demdiff_2016-2014.nc'

# list of ICESat-2 
masked_IS2 = directory_spider(data_dir, path_pattern='ICESat', file_pattern='.pkl')
masked_IS2 = [f for f in masked_IS2 if '._' not in f]

In [11]:
dh_min = -20
dh_max = 7
ddh = 3
contour_levels = np.arange(0, dh_max + ddh, ddh)
scmap = shifted_colormap(cmc.vik, midpoint=1 - (dh_max) / (dh_max + abs(dh_min)))

In [9]:
REMA_file = os.path.join(REMA_dir, LagrangianDiff_file)
is2_files = [[file for file in masked_IS2 if f'/2022/' in file and f'/0401/' in file][0],
             [file for file in masked_IS2 if f'/2022/' in file and f'/0470/' in file][0],
             [file for file in masked_IS2 if f'/2018/' in file and f'/0843/' in file][0],
             [file for file in masked_IS2 if f'/2018/' in file and f'/0843/' in file][0],
            ]
track_beam = (['0401', '0470', '0843', '0843'], ['L', 'R', 'L','R'])
gts = ['GT1L','GT1R','GT3L', 'GT3R']

# modify bounds for fitting
bounds = BOUNDS.copy()
# Young's Modulus, E (Pa)
bounds[:, 0] = [1e9, 10e9]
# Ice shelf thickness, H (m)
bounds[:,1] = [10, 1e3]
# Lake radius, Ld (m)
bounds[:,2] = [100, 1300]

initial_guess = [5e9, 868, 100, 20]

In [7]:
# define some useful functions
# residual sum of squares
def rss(obs, pred):
    return np.sum((obs - pred)**2)

# convert the "predicted" values to 
def C2S(E, H, Ld, d):
    return 'E = {0:0.02f} MPa\nH = {1:0.02f} m\nLd = {2:0.02f} m\nd = {3:0.02f}'.format(E/1e6, H, Ld, d)

# deflection function to minimize
def f(r, E, H, Ld, d):
    return deflection(r, 0, E, H, Ld, d)

In [14]:
NPROFS = 4

SAVE = False
fig_save_dir = 'figs'
fig_type = '.svg'
save_name = 'figure3'

figsize = [13.504,  8.0356]

##########################################################################################
# create all subplots and gridspecs first
##########################################################################################
plt.close('all')
fig = plt.figure(num=1,clear=1,figsize=figsize)

PARENT = gridspec.GridSpec(1,2, figure=fig, 
                           width_ratios=[1.,1.33])

gs0 = gridspec.GridSpecFromSubplotSpec(1,1, subplot_spec=PARENT[0], wspace=0.03)
demax = fig.add_subplot(gs0[0])

gs1 = gridspec.GridSpecFromSubplotSpec(NPROFS,1, subplot_spec=PARENT[1], hspace=0.01)
sax = [fig.add_subplot(gs1[i]) for i in range(NPROFS)]

##########################################################################################
# use dem context
##########################################################################################

ddm = xr.open_dataset(REMA_file)

left, right = ddm.x.values.min(), ddm.x.values.max()
bottom, top = ddm.y.values.min(), ddm.y.values.max()

dh = ddm.dh.values
dh[dh < -100] = np.nan
dh[dh > 100] = np.nan

p = demax.imshow(dh, 
                  cmap=scmap, vmin=dh_min, vmax=dh_max,
                 extent=(left, right, bottom, top), rasterized=True, )
demax.set(xticklabels=[], yticklabels=[],
             xticks=[], yticks=[],
             xlim=(left, right), ylim=(bottom, top),
         )

cfs = demax.contour(ddm.x, ddm.y, ddm.dh, 
              levels=contour_levels, 
              colors=['k'], linewidths=[0.5], alpha=0.66)

ll=xy2ll(left, bottom), xy2ll(left, top), xy2ll(right, top), xy2ll(right, bottom)
ll=np.array(ll)
lon_range = [ll[:,0].min(), ll[:,0].max()]
lat_range = [ll[:,1].min(), ll[:,1].max()]
demax = plot_latlon_lines(demax, lat_range=lat_range, lon_range=lon_range,
                          extents=(left, right, bottom, top),
                          )


inset = demax.inset_axes([0.45, 0.15, 0.433, 0.07])
inset.xaxis.set_ticks_position('bottom')

scalebar = AnchoredSizeBar(demax.transData,
                           1e3, '1 km', 'lower left', 
                           pad=1,
                           frameon=False,
                           size_vertical=100,
                          )

demax.add_artist(scalebar)


cb = fig.colorbar(p, cax=inset, 
                  label='Elevation difference $\Delta h$ (m)',
                  orientation='horizontal', extend='both')
cb.set_ticks([dh_min, dh_min/2, 0, dh_max])

##########################################################################################
# do the is2 tracks and fitting now
##########################################################################################
ylims = []
xlims = []

xticks = [-72.05 - 0.025, -72.05, -72.025, -72.0, -72 + 0.025]
xticklabels = ['{0}°'.format(f'{v:0.2f}')  for i,v in enumerate(xticks)]
letter = ['\b(b)', '(c)', '(d)', '(e)']

for i, file in enumerate(is2_files):
    # sample REMA DEM along ICESat-2 track and fit to analytical deflection solution
    ans = fitting_REMA_dh(file, REMA_file,
                          center_x=None, center_y=None,
                          bounds=bounds,
                          gt='gt3' if '/0843/' in file else 'gt1',
                          beam=track_beam[1][i],
                          initial_guess=initial_guess)
    x_is2, y_is2, lat, lon, h_is2, r, dh_rema, uplift, surface, C = ans
    demax.plot(x_is2, y_is2, c=COLS[i], ls='-')

    E,H,Ld,d = C
    print(C2S(*C))
    print('estimated volume = ', np.pi * Ld**2 * d / 1e6, 'x10^6 m^3')
    
    sax[i].text(0.03, 0.2, f'{letter[i]} Track {track_beam[0][i]} {gts[i]}', color=COLS[i],
                va='center', ha='left', fontsize=16,
                backgroundcolor='#ffffffe4',
                transform=sax[i].transAxes,
               )
    sax[i].axvspan(lat[np.isclose(r, Ld, atol=10)][-1], lat[np.isclose(r, -Ld, atol=10)][-1],
                   label='Lake extent',
                  fc='k', ec='none', alpha=0.1)
    sax[i].plot(lat, dh_rema, c=COLS[i])
    sax[i].plot(lat, uplift, c='k', ls='--', label='Model uplift')
    sax[i].plot(lat, surface, c='k', label='Model surface')
    
    if i == 1:
        sax[i].legend(loc='lower right', 
                      frameon=True, edgecolor='none',facecolor='w')
    sax[i].set(ylabel=r'$\Delta h$ (m)')
    
    if i == len(is2_files) - 1:
        sax[i].set(xlabel='Latitude')
    
    xlims.append([lat.min(), lat.max()])
    ylims.append(sax[i].get_ylim())
    
    sax[i].set_xticks(xticks)
    [sax[i].axvline(v, c='k', ls='--', lw=0.3) for v in [-72.05, -72.0]]
    
    if i < NPROFS - 1:
        sax[i].set(xticklabels=[])
    else:
        sax[i].set_xticklabels(xticklabels)
    
    print('  fit rss:', rss(dh_rema, surface))

for ax in sax:
     ax.set(ylim=(np.min(ylims), np.max(ylims)),
            yticks=[-20, -10, 0, 10],
            xlim=(np.min(xlims), np.max(xlims)))
if SAVE:
    plt.savefig(os.path.join(fig_save_dir, save_name + fig_type), dpi=600)

260 35
E = 1709.47 MPa
H = 124.07 m
Ld = 792.51 m
d = 22.31
estimated volume =  44.030467443146485 x10^6 m^3
  fit rss: 5452.260252094623
238 50
E = 1603.72 MPa
H = 111.44 m
Ld = 803.55 m
d = 19.61
estimated volume =  39.78037549111451 x10^6 m^3
  fit rss: 4742.403199494797
262 45
E = 1709.53 MPa
H = 120.03 m
Ld = 870.95 m
d = 21.63
estimated volume =  51.54271289406852 x10^6 m^3
  fit rss: 3744.6804114281517
259 43
E = 1719.99 MPa
H = 121.24 m
Ld = 870.80 m
d = 20.90
estimated volume =  49.78210523107446 x10^6 m^3
  fit rss: 3735.611909027797
